In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pickle

from gpu import set_gpu
import copy as cp
from sklearn.utils import shuffle

import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from DL_utils import MLP, MLPbeta, CNN, eval_nn, get_train, get_test
import session_new as session

import process_data as prd
from lda import train_lda, predict, eval_lda, eval_lda_ch

set_gpu()

Using TensorFlow backend.


Num GPUs Available:  1


In [2]:
sub_type = 'TR'
with open('train_data_raw_'  + sub_type + '.p', 'rb') as f:
    raw, params,_,_ = pickle.load(f)

n_subs = np.max(params[:,0])

In [3]:
train_sess = {'sub_type':sub_type,'train':'fullallmix4', 'train_grp':2, 'train_scale':5, 'cv_type':'all','scaler_load':False,'feat_type':'feat','epochs':30}
test_sess = {'test_grp':4, 'test':'partposrealmixeven24'}
sess = session.Sess(**train_sess)
sess.update(**test_sess)
n_subs = 4#np.max(params[:,0])

for sub in range(2,n_subs+1):
    sess.sub = sub
    # tf.keras.backend.clear_session()
    train_ind = (params[:,0] == sess.sub) & (params[:,3] == sess.train_grp)
    test_ind = (params[:,0] == sess.sub) & (params[:,3] == sess.test_grp)

    n_dof = np.max(params[train_ind,4])
    
    # Train NNs
    mlp = MLP(n_class=n_dof)
    mlp_beta = MLPbeta(n_class=n_dof)
    cnn = CNN(n_class=n_dof)

    optimizer = tf.keras.optimizers.Adam()

    train_loss = tf.keras.metrics.Mean(name='train_loss')
    train_accuracy = tf.keras.metrics.CategoricalAccuracy(name='train_accuracy')
    
    if np.sum(train_ind) and np.sum(test_ind):
        trainmlp, validmlp, traincnn, validcnn, y_train, y_valid, x_train_mlp, x_train_cnn, x_train_lda, y_train_lda, x_train_aug, y_train_aug = prd.prep_train_data(sess,raw,params)

        # with open('noisedata_all_emgscalelim/AB' + str(sub) + '_grp_2_fullallmix4_5.p','rb') as f:
        #     scaler, x_train_cnn, _, _, _, y_train, _, x_train_lda, y_train_lda, x_train_aug, y_train_aug = pickle.load(f)
        
        # x_train_cnn = x_train_cnn.astype('float32')
        # x_train_mlp = x_train_cnn.reshape(x_train_cnn.shape[0],-1)
        
        # trainmlp = tf.data.Dataset.from_tensor_slices((x_train_mlp, y_train)).shuffle(x_train_mlp.shape[0],reshuffle_each_iteration=True).batch(128)
        # traincnn = tf.data.Dataset.from_tensor_slices((x_train_cnn, y_train)).shuffle(x_train_cnn.shape[0],reshuffle_each_iteration=True).batch(128)
        
        # sess.scaler = scaler

        models = [mlp, mlp_beta, cnn]

        for model in models:
            # print('Training ' + str(model))

            if isinstance(model,CNN):
                ds = traincnn
            else:
                ds = trainmlp
            
            train_mod = get_train()

            for epoch in range(sess.epochs):
                # Reset the metrics at the start of the next epoch
                train_loss.reset_states()
                train_accuracy.reset_states()

                for x, y in ds:
                    train_mod(x, y, model, optimizer, train_loss, train_accuracy)

                if epoch == 0 or epoch == sess.epochs-1:
                    print(
                        f'Epoch {epoch + 1}, '
                        f'Loss: {train_loss.result():.2f}, '
                        f'Accuracy: {train_accuracy.result() * 100:.2f} '
                    )
            
            del train_mod
        
        # Train aligned
        y_train_aligned = np.argmax(y_train, axis=1)[...,np.newaxis]

        mlp_enc = mlp.get_layer(name='enc')
        w_mlp, c_mlp,_, _, _ = train_lda(mlp_enc(x_train_mlp).numpy(),y_train_aligned)

        mlpbeta_enc = mlp_beta.get_layer(name='enc')
        w_mlpbeta, c_mlpbeta,_, _, _ = train_lda(mlpbeta_enc(x_train_mlp).numpy(),y_train_aligned)
        
        cnn_enc = cnn.get_layer(name='enc')
        temp = cnn_enc(x_train_cnn[:x_train_cnn.shape[0]//2,...]).numpy()
        temp2 = np.vstack((temp,cnn_enc(x_train_cnn[x_train_cnn.shape[0]//2:,...]).numpy()))
        w_cnn, c_cnn,_, _, _ = train_lda(temp2,y_train_aligned)

        # Train LDA
        w,c, _, _, _ = train_lda(x_train_lda,y_train_lda)
        w_aug,c_aug, _, _, _ = train_lda(x_train_aug,y_train_aug)

        # Load test data
        with open('real_noise/all_real_noise.p', 'rb') as f:
            real_noise_temp, _ = pickle.load(f)

        x_test_cnn, x_test_mlp, x_test_lda, y_test, clean_size = prd.prep_test_data(sess, raw, params, real_noise_temp)

        with open('testdata_all_emgscalelim_noisescalelim/TR' + str(sub) + '_grp_2_partposrealmixeven14_1.p','rb') as f:
            x_test_cnn, _, x_test_lda, y_test = pickle.load(f) 
        x_temp = np.transpose(x_test_lda.reshape((x_test_lda.shape[0],4,-1)),(0,2,1))[...,np.newaxis]
        x_test_cnn = sess.scaler.transform(x_temp.reshape(x_temp.shape[0]*x_temp.shape[1],-1)).reshape(x_temp.shape)
        x_test_cnn = x_test_cnn.astype('float32')

        # Reshape for nonconvolutional SAE
        x_test_mlp = x_test_cnn.reshape(x_test_cnn.shape[0],-1)

        # Test
        mlp_test_aligned = mlp_enc(x_test_mlp).numpy()
        mlpbeta_test_aligned = mlpbeta_enc(x_test_mlp).numpy()
        cnn_test_aligned = cnn_enc(x_test_cnn).numpy()
        y_test_aligned = np.argmax(y_test, axis=1)[...,np.newaxis]

        clean_lda = eval_lda(w, c, x_test_lda[:clean_size,...], y_test_aligned[:clean_size,...])
        noisy_lda = eval_lda(w, c, x_test_lda[clean_size:,...], y_test_aligned[clean_size:,...])
        print(
            f'LDA ---- '
            f'Clean: {clean_lda * 100:.2f}, '
            f'Noisy: {noisy_lda * 100:.2f}'
        )

        clean_lda = eval_lda(w_aug, c_aug, x_test_lda[:clean_size,...], y_test_aligned[:clean_size,...])
        noisy_lda = eval_lda(w_aug, c_aug, x_test_lda[clean_size:,...], y_test_aligned[clean_size:,...])
        print(
            f'AUG ---- '
            f'Clean: {clean_lda * 100:.2f}, '
            f'Noisy: {noisy_lda * 100:.2f}'
        )

        clean_acc, noisy_acc = eval_nn(x_test_mlp, y_test,mlp,clean_size)
        clean_lda = eval_lda(w_mlp,c_mlp,mlp_test_aligned[:clean_size,...],y_test_aligned[:clean_size,...])
        noisy_lda = eval_lda(w_mlp,c_mlp,mlp_test_aligned[clean_size:,...],y_test_aligned[clean_size:,...])
        print(
            f'MLP ---- '
            f'Clean: {clean_acc * 100:.2f}, '
            f'Noisy: {noisy_acc * 100:.2f}, '
            f'LDA Clean: {clean_lda * 100:.2f}, '
            f'LDA Noisy: {noisy_lda * 100:.2f}'
        )

        clean_acc, noisy_acc = eval_nn(x_test_mlp, y_test,mlp_beta,clean_size)
        clean_lda = eval_lda(w_mlpbeta,c_mlpbeta,mlpbeta_test_aligned[:clean_size,...],y_test_aligned[:clean_size,...])
        noisy_lda = eval_lda(w_mlpbeta,c_mlpbeta,mlpbeta_test_aligned[clean_size:,...],y_test_aligned[clean_size:,...])
        print(
            f'MLPB ---- '
            f'Clean: {clean_acc * 100:.2f}, '
            f'Noisy: {noisy_acc * 100:.2f}, '
            f'LDA Clean: {clean_lda * 100:.2f}, '
            f'LDA Noisy: {noisy_lda * 100:.2f}'
        )

        clean_acc, noisy_acc = eval_nn(x_test_cnn, y_test,cnn,clean_size)
        clean_lda = eval_lda(w_cnn,c_cnn,cnn_test_aligned[:clean_size,...],y_test_aligned[:clean_size,...])
        noisy_lda = eval_lda(w_cnn,c_cnn,cnn_test_aligned[clean_size:,...],y_test_aligned[clean_size:,...])
        print(
            f'CNN ---- '
            f'Clean: {clean_acc * 100:.2f}, '
            f'Noisy: {noisy_acc * 100:.2f}, '
            f'LDA Clean: {clean_lda * 100:.2f}, '
            f'LDA Noisy: {noisy_lda * 100:.2f}'
        )
    else:
        print('no training or testing data')



Loading training data: traindata_all/TR2_traindata_2.p
Epoch 1, Loss: 1.93, Accuracy: 21.52 
Epoch 30, Loss: 0.64, Accuracy: 73.27 
Epoch 1, Loss: 1.11, Accuracy: 57.60 
Epoch 30, Loss: 0.25, Accuracy: 89.96 
Epoch 1, Loss: 0.97, Accuracy: 65.59 
Epoch 30, Loss: 0.22, Accuracy: 91.15 
Loading training data: traindata_all/TR2_traindata_4.p
LDA ---- Clean: 8.11, Noisy: 8.98
AUG ---- Clean: 23.32, Noisy: 22.81
MLP ---- Clean: 21.18, Noisy: 22.64, LDA Clean: 21.04, LDA Noisy: 22.55
MLPB ---- Clean: 13.57, Noisy: 14.71, LDA Clean: 13.89, LDA Noisy: 15.17
CNN ---- Clean: 22.32, Noisy: 21.33, LDA Clean: 23.86, LDA Noisy: 22.57
Loading training data: traindata_all/TR3_traindata_2.p
Epoch 1, Loss: 1.68, Accuracy: 27.23 
Epoch 30, Loss: 0.91, Accuracy: 63.80 
Epoch 1, Loss: 1.15, Accuracy: 51.76 
Epoch 30, Loss: 0.47, Accuracy: 81.88 
Epoch 1, Loss: 1.10, Accuracy: 54.81 
Epoch 30, Loss: 0.39, Accuracy: 84.72 
Loading training data: traindata_all/TR3_traindata_4.p
LDA ---- Clean: 6.55, Noisy: 7.

KeyboardInterrupt: 

In [ ]:
Loading training data: traindata_all/AB2_traindata_2.p
Loading training data: traindata_all/AB2_traindata_4.p
LDA ---- Clean: 90.23, Noisy: 58.54
AUG ---- Clean: 56.60, Noisy: 53.51
MLP ---- Clean: 79.37, Noisy: 75.31, LDA Clean: 79.60, LDA Noisy: 75.63
MLPB ---- Clean: 92.00, Noisy: 84.43, LDA Clean: 92.51, LDA Noisy: 85.51
CNN ---- Clean: 88.51, Noisy: 83.14, LDA Clean: 91.06, LDA Noisy: 84.77
Loading training data: traindata_all/AB3_traindata_2.p
Loading training data: traindata_all/AB3_traindata_4.p
LDA ---- Clean: 76.14, Noisy: 60.57
AUG ---- Clean: 57.60, Noisy: 51.94
MLP ---- Clean: 73.09, Noisy: 70.29, LDA Clean: 73.94, LDA Noisy: 71.40
MLPB ---- Clean: 72.43, Noisy: 71.23, LDA Clean: 70.54, LDA Noisy: 69.09
CNN ---- Clean: 70.94, Noisy: 70.60, LDA Clean: 69.80, LDA Noisy: 69.31
Loading training data: traindata_all/AB4_traindata_2.p
Loading training data: traindata_all/AB4_traindata_4.p
LDA ---- Clean: 86.49, Noisy: 65.66
AUG ---- Clean: 51.40, Noisy: 48.03
MLP ---- Clean: 83.97, Noisy: 80.17, LDA Clean: 84.34, LDA Noisy: 79.03
MLPB ---- Clean: 80.14, Noisy: 78.69, LDA Clean: 81.17, LDA Noisy: 79.80
CNN ---- Clean: 81.83, Noisy: 81.11, LDA Clean: 81.86, LDA Noisy: 80.69

In [ ]:
Loading training data: traindata_all/AB2_traindata_2.p
Loading training data: traindata_all/AB2_traindata_4.p
LDA ---- Clean: 90.23, Noisy: 58.54
AUG ---- Clean: 56.80, Noisy: 53.34
MLP ---- Clean: 89.29, Noisy: 82.91, LDA Clean: 90.11, LDA Noisy: 84.17
MLPB ---- Clean: 87.31, Noisy: 80.71, LDA Clean: 89.03, LDA Noisy: 82.06
CNN ---- Clean: 91.20, Noisy: 83.46, LDA Clean: 92.40, LDA Noisy: 85.49
Loading training data: traindata_all/AB3_traindata_2.p
Loading training data: traindata_all/AB3_traindata_4.p
LDA ---- Clean: 76.14, Noisy: 60.57
AUG ---- Clean: 57.60, Noisy: 52.34
MLP ---- Clean: 68.54, Noisy: 68.69, LDA Clean: 70.91, LDA Noisy: 70.14
MLPB ---- Clean: 76.80, Noisy: 75.89, LDA Clean: 77.40, LDA Noisy: 76.97
CNN ---- Clean: 72.26, Noisy: 71.37, LDA Clean: 69.54, LDA Noisy: 69.46
Loading training data: traindata_all/AB4_traindata_2.p
Loading training data: traindata_all/AB4_traindata_4.p
LDA ---- Clean: 86.49, Noisy: 65.66
AUG ---- Clean: 51.11, Noisy: 48.06
MLP ---- Clean: 85.09, Noisy: 80.69, LDA Clean: 84.29, LDA Noisy: 80.00
MLPB ---- Clean: 78.34, Noisy: 76.71, LDA Clean: 82.09, LDA Noisy: 79.51
CNN ---- Clean: 81.86, Noisy: 80.63, LDA Clean: 83.71, LDA Noisy: 81.80